# Clean the data and make parquet files

 The files we were given for the northwest region from replica have header rows embedded through out due to the way the google cloud on their end stacks things.

 Here we clean convert every column to strings and then remove those rows and save the data to parquet files which are easier to deal with.  



In [1]:
%load_ext autoreload
%autoreload 2
from replicaEVSE import datautils as du
import os
import dask.dataframe as dd
import pandas as pd

datadir = '../../data/'

In [3]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8)  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default

client

/home/matthew.wilde@RMI.local/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39097 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39097/status,
Dashboard: http://127.0.0.1:39097/status,Workers: 8
Total threads: 64,Total memory: 251.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38465,Workers: 8
Dashboard: http://127.0.0.1:39097/status,Total threads: 64
Started: Just now,Total memory: 251.88 GiB
Comm: tcp://127.0.0.1:45703,Total threads: 8
Dashboard: http://127.0.0.1:34985/status,Memory: 31.49 GiB
Nanny: tcp://127.0.0.1:36865,


### path to the data

In [3]:
datapath = '../../data/'
popfile = 'northwest_2021_Q4_population.csv'
tripsatfile = 'northwest_2021_Q4_saturday_trip.csv'
tripthufile = 'northwest_2021_Q4_thursday_trip.csv'

### load and convert all the data to strings

In [ ]:
pop_df = du.load_data(os.path.join(datapath, popfile))
tripsat_df = du.load_data(os.path.join(datapath, tripsatfile))
tripthu_df = du.load_data(os.path.join(datapath, tripthufile))

### clean the data of bad rows (embedded headers in the data) and return the dask data frames for inspection.

In [ ]:
pop_df = du.clean_pop_data(pop_df)
tripsat_df = du.clean_trip_data(tripsat_df)
tripthu_df = du.clean_trip_data(tripthu_df)

### Or we can use our wrapper to clean save the data in the parquet format to speed analysis later

In [ ]:
popparquet = 'northwest_2021_Q4_population.parquet'
tripsatparquet = 'northwest_2021_Q4_saturday_trip.parquet'
tripthuparquet = 'northwest_2021_Q4_thursday_trip.parquet'
pop_df = pop_df.to_parquet(os.path.join(datapath, popparquet))
tripsat_df = tripsat_df.to_parquet(os.path.join(datapath, tripsatparquet))
tripthu_df = tripthu_df.to_parquet(os.path.join(datapath, tripthuparquet))

In [ ]:
popparquet = 'northwest_2021_Q4_population.parquet'
tripthuparquet = 'northwest_2021_Q4_thursday_trip.parquet'
df = dd.read_parquet(os.path.join(datapath, tripthuparquet))

# Make a joined table and save as parquet

In [4]:
# get list of blockgroups
gdf = pd.read_pickle(datadir+'/blockgroup_boundaries.pkl')
bgrp_list = list(gdf.GEOID.values)

trip_sat_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_saturday_trip.parquet')
trip_thu_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_thursday_trip.parquet')

pop_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_population.parquet')
# pop_ddf = dd.merge(pop_ddf, counties, on='person_id', how='left')


trip_sat_ddf['weekday'] = 'saturday'
trip_thu_ddf['weekday'] = 'thursday'

# stack the two dataframes
trips = dd.concat([trip_sat_ddf, trip_thu_ddf], axis=0, keys=["saturday", "thursday"])


# only trips that end in WA
trips_ddf = trips.loc[trips['destination_bgrp'].isin(bgrp_list)]
 
merged_ddf = dd.merge(trips_ddf, pop_ddf, on='person_id', how='left')

# Create charge_type column from travel_purpose column
merged_ddf['charge_type'] = merged_ddf.travel_purpose

merged_ddf['charge_type'] = merged_ddf.travel_purpose
merged_ddf['charge_type'] = merged_ddf['charge_type'].where(
    merged_ddf.charge_type.isin(
    ['WORK', 'HOME']), 'PUBLIC')



In [5]:
# sort the table before saving it. needs to be sorted by person_id most critically
# since we are then chunking the table. If they are not near each other we will get
# some funky results. 
merged_ddf = merged_ddf.sort_values(by=['person_id', 'start_time', 'weekday']).reset_index(drop=True)

: 

: 

In [ ]:
merged_ddf.to_parquet(datadir+'/wa_pop_and_trips.parquet', overwrite=True)

In [20]:
dtype_dict = {"person_id": str, "home_cty": str, "home_st": str}
counties = dd.read_csv(datadir+'replica-wa_tes-04_22_23-population_dataset.csv', dtype=dtype_dict)
counties.to_parquet(datadir+'population_counties_dataset.parquet', engine='pyarrow', overwrite=True)

### make a table of just trips into WA

In [3]:

# get list of blockgroups
gdf = pd.read_pickle(datadir+'/blockgroup_boundaries.pkl')
bgrp_list = list(gdf.GEOID.values)

trip_sat_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_saturday_trip.parquet')
trip_thu_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_thursday_trip.parquet')

# dtype_dict = {"person_id": str, "home_cty": "category", "home_st": "category"}
dtype_dict = {"person_id": str, "home_cty": str, "home_st": str}
counties = dd.read_parquet(datadir+'/population_counties_dataset.parquet', engine='pyarrow')

trip_sat_ddf['weekday'] = 'saturday'
trip_thu_ddf['weekday'] = 'thursday'

# stack the two dataframes
trips = dd.concat([trip_sat_ddf, trip_thu_ddf], axis=0, keys=["saturday", "thursday"])


# only trips that end in WA
trips_ddf = trips.loc[trips['destination_bgrp'].isin(bgrp_list)]

In [4]:
trips_into_wa_len = len(trips_ddf) # = 51727268

In [8]:
merged_ddf = dd.read_parquet(datadir+'/wa_pop_and_trips.parquet')
len(merged_ddf)

51727268

In [9]:
len(trips_ddf['person_id'].unique()) # = 8538399

8538399

In [12]:
merged_ddf['charge_type'].value_counts().compute()

PUBLIC    31224030
HOME      16301817
WORK       4201421
Name: charge_type, dtype: int64

# made a commercial only table

In [6]:
# get list of blockgroups
gdf = pd.read_pickle(datadir+'/blockgroup_boundaries.pkl')
bgrp_list = list(gdf.GEOID.values)

trip_sat_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_saturday_trip.parquet')
trip_thu_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_thursday_trip.parquet')

pop_ddf = dd.read_parquet(datadir+'/northwest_2021_Q4_population.parquet')
# pop_ddf = dd.merge(pop_ddf, counties, on='person_id', how='left')


trip_sat_ddf['weekday'] = 'saturday'
trip_thu_ddf['weekday'] = 'thursday'

# stack the two dataframes
trips = dd.concat([trip_sat_ddf, trip_thu_ddf], axis=0, keys=["saturday", "thursday"])


# only trips that end in WA
trips_ddf = trips.loc[trips['destination_bgrp'].isin(bgrp_list)]


trips_ddf = trips_ddf.loc[trips_ddf['mode'] == 'COMMERCIAL'].reset_index(drop=True)

In [7]:
trips_ddf.head()

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,...,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday
0,8729929711006647766,9261044664253989731,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 16:54:44,0 days 22:11:48,360.828959,HEAVY_COMMERCIAL,060930009002,...,nan,nan,nan,nan,nan,41.45456,-122.44331,45.68459,-122.61634,saturday
1,603083185441061207,11705321199965034957,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 15:54:04,0 days 17:15:06,67.862579,MEDIUM_COMMERCIAL,160099501002,...,nan,nan,nan,nan,nan,47.31601,-116.56562,47.63069,-117.54066,saturday
2,12605203164736745787,12047641940475444253,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 10:35:44,0 days 11:44:52,57.083123,MEDIUM_COMMERCIAL,160099501002,...,nan,nan,nan,nan,nan,47.31601,-116.57249,47.65752,-117.33329,saturday
3,10073082753102857121,16409299978104629971,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 12:58:57,0 days 13:11:16,8.043207,MEDIUM_COMMERCIAL,160550003011,...,nan,nan,nan,nan,nan,47.74532,-117.00920,47.69913,-117.14790,saturday
4,12045057915234869999,13147923906478375057,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 06:40:10,0 days 07:09:20,21.660744,MEDIUM_COMMERCIAL,160550003022,...,nan,nan,nan,nan,nan,47.80623,-116.88697,47.65752,-117.23854,saturday


In [9]:
datadir+'wa_trucks.parquet'

'../../data/wa_trucks.parquet'

In [10]:
trips_ddf.to_parquet(datadir+'/wa_trucks.parquet', overwrite=True)

In [11]:
len(trips_ddf)

1452450

In [12]:
client.close()